In [1]:
#Importing the libraries
%matplotlib inline
from scipy.spatial import distance
import random
import numpy as np
import matplotlib.pyplot as plt
#import plotly
from scipy.spatial import distance
import time

In [2]:
## generating Random 100 points/nodes in a plane ##
def N_Points_generator(n):
  np.random.seed(0)
  random_float_array = np.random.uniform(1.0, 10.0, size=(n, 2))
  return random_float_array

In [3]:
### Computing One_Hop Neighbours of Node #####
def One_Hop_neighbours(random_float_array,dict1):
  # one hop neighbours for nodes:
  #dict = dict1
  array_one_hop = random_float_array
  print(len(array_one_hop))
  for i in range(len(array_one_hop)):
#random_float_array[i][2] = []
    test = array_one_hop[i]
    exp_array = np.delete(array_one_hop,i,0)
    one_hops = []
    for j in exp_array:

      d = distance.euclidean(j,test)
      if d <= 1:
        one_hops.append(j)

    dict1[str(random_float_array[i])] =  one_hops 
  return dict1
  
  

In [4]:
### Adding Source Node to the Data #####
def Add_Source_Node(x_cordinate, y_cordinate,dict1,array_one_hop):
  li =[]
  s = np.array([x_cordinate,y_cordinate])
  for i in array_one_hop:

      d = distance.euclidean(i,s)
      if d <=1:
        li.append(i)

  dict1[str(s)] = li
  return dict1,s

In [5]:
#### Get one hops of a Node #####
def get_one_hops(key,dict1):
  return dict1[str(key)]

In [6]:
#### plot/see the simulation graphs in each step how flooding works and how RBF do coverage #########
def plot_interface(s,r,list,retransmission,count):
  fig = plt.figure()
  #fig = plt.figure(figsize=(10, 10))
  colors = (0,0,0)
  area = np.pi*3
  plt.scatter(random_float_array[:,0],random_float_array[:,1], alpha=0.5)
  plt.title('Flooding with RDF algo In a 10x10 grid'+ count)
  plt.xlabel('x')
  plt.ylabel('y')
  #plt.show()
 
  for i in retransmission:
    plt.scatter(retransmission[0],retransmission[1],color='red')
  for i in list:
    plt.scatter(i[0],i[1],color='orange')
  plt.scatter(1.5,1.5,color='red')
  circle = plt.Circle((1.5, 1.5), radius=1, fc='y', linewidth=2,fill=False,color='red')
  plt.gca().add_patch(circle)
  
  plt.scatter(s[0],s[1] ,color='green')
  circle = plt.Circle((s[0],s[1]), radius=1, fc='y', linewidth=0.5,fill=False,color='green')
  plt.gca().add_patch(circle)
  plt.scatter(r[0],r[1],color='black')
  circle = plt.Circle((r[0],r[1]), radius=1, fc='y', linewidth=0.5,fill=False,color='black')
  plt.gca().add_patch(circle)
  

  #fig.canvas.draw()
  #display(fig)
  #plt.show()
  return fig
  #display.clear_output(wait=True)
  #display.display(plt.gcf())
  #fig.canvas.draw()
  #display(fig)
  

In [7]:
##### Code for Diviving into Groups or Finding receiver and sender coverage nodes that are common #########################
def intersect(a, b):
    """ return the intersection of two lists """
    
    l2 = [ i  for i in a if i in np.array(b)]
    l3 = [ i  for i in b if i not in np.array(a)]
   
    return l2,l3

In [8]:
####### The RDF ALDO ################
def RBF_algo(s_one_hop,r_one_hop):
  #find the intersection neighbour's
  inter, inrecer = intersect(s_one_hop,r_one_hop)
  #print("intesection",inter)
  #print(" out of intersection",inrecer)
  #find the neighbours that are out of source and in the reciever range
  if len(inter) > 0:
    for i in inter:
      cover =False
      for j in inrecer:
        if distance.euclidean(i,j) <=1:
          cover = True
          break
      if (not cover):
        return False
    return True
  else:
    return False
        
        

In [9]:
#### Utlity Function For Flooding #######
def near2rec(rec):
  list1 =[]
  for i in random_float_array:
    a = 10
    d = distance.euclidean(rec,i)
    if  d > 1:
      if a>d:
          a = d
          list1.append(i)

  return list1[len(list1)-1]

In [10]:
#### Utiltiy Function For Flooding #######
def retransmission(source_one_hop,dict1):
  receiverl = []
  for node in s_one_hops:
      #receiver = random.choice(s_one_hops)
      receiver = node
      if receiver not in np.array(msg_recived_list):
        print("receiver node",receiver)
        r_one_hops = get_one_hops(receiver,dict1)
        print("print of receiver one hops",r_one_hops)
        # remove if there is source node
        wo_r_one_hops  = [i for i in r_one_hops if i not in np.array(source)]
        #if len(wo_r_one_hops) == 0:
        #  source = near2rec(receiver)
        #  print(source)
        #  break
        #print("reciever one hop out of msglist",wo_r_one_hops)
        #plot_interface(source,receiver,msg_recived_list)
        #display.clear_output(wait=True)
        #time.sleep(1.0)
        #msg_recived_list.append(receiver)
        fr = RBF_algo(s_one_hops,wo_r_one_hops)
        #print(fr)
        if (not fr):
          receiverl.append(receiver)
          #break
  return receiverl

In [11]:
###### Utility Function For Flooding #######
def near_source(msg_recived_list,reciver,dict1,random_float_array):
  dict2 ={}
  for i,j in enumerate(random_float_array):
    #print(i)
    if j not in np.array(msg_recived_list):
      d = distance.euclidean(reciver,j)
      if d > 1:
        dict2[i] = d
 
  #print(dict2)
  
  temp = min(dict2.values()) 
  #print(temp)
  res = [key for key in dict2 if dict2[key] == temp]
  #print(res)
  #print(random_float_array[res])
  return random_float_array[res[0]]     

In [12]:
#### Main driver Of the code #########
"""
Use this cell or this as a main function for the file, To see the step by step process of flodding inferentially please uncooment the plot interface function/ method 
that plots figures step wise. (i.e line 60 & 61)

"""
if __name__ == "__main__": 
  %matplotlib inline
#from IPython import display
random.seed(5)
msg_recived_list = []
retrans_l = []
forwarding_nodes = []
dict1 = {}
count = 0
random_float_array = N_Points_generator(100)

array_one_hop = random_float_array

dict1 = One_Hop_neighbours(random_float_array,dict1)

dict1,s = Add_Source_Node(1.5,1.5,dict1,array_one_hop)

source = s
#print(dict1)

while (len(msg_recived_list) <= 99):
  
  s_one_hops = get_one_hops(source, dict1)
  forwarding_nodes.append(source)
  #print("source",source)
  if (len(s_one_hops)==1):
    #print ('msg transmittesd to its only one-hop and added it into the list')
    if s_one_hops[0] not in np.array(msg_recived_list):
      msg_recived_list.append(s_one_hops[0])
      source = s_one_hops[0]
    else:
      source = near_source(msg_recived_list,source,dict1,random_float_array)
      #print("morach tisk with basics")
      #print(source)
      #break
      
      
   # msg_recived_list.append(s_one_hops[0])
    #print('updating receiver as source node')
    #source = s_one_hops[0]
    
  else:
   
    #print("source one hop",s_one_hops)
    receiverl = []
    for node in s_one_hops:
        #receiver = random.choice(s_one_hops)
        receiver = node
        
        if receiver not in np.array(msg_recived_list):
        #if receiver not in np.array(msg_recived_list):
          msg_recived_list.append(receiver)
          #print("msg list",msg_recived_list)
          #print("receiver node",receiver)
          r_one_hops = get_one_hops(receiver,dict1)
          #print("print of receiver one hops",r_one_hops)
          # remove if there is source node
          wo_r_one_hops  = [i for i in r_one_hops if i not in np.array(source)]
          #if len(wo_r_one_hops) == 0:
          #  source = near2rec(receiver)
          #  print(source)
          #  break
          #print("reciever one hop out of msglist",wo_r_one_hops)
          #xtest = plot_interface(source,receiver,msg_recived_list,[],str(count))
          #xtest.show()
          count = count+1
          #exit()
          
          #display.clear_output(wait=True)
          #display.display(pl.gcf())
          time.sleep(1)
          #msg_recived_list.append(receiver)
          fr = RBF_algo(s_one_hops,wo_r_one_hops)
          #print(fr)
          if (not fr):
            receiverl.append(receiver)
            retrans_l.append(receiverl)
    
    if len(receiverl)>=1:
     # print(receiverl)
      rettrans = []
      for i in receiverl:
        one_h = get_one_hops(i,dict1)
        trans = [i for i in one_h if i not in np.array(msg_recived_list)]
        if len(trans) >= 1:  
          rettrans.extend(trans)
      if len(rettrans)>=1:
        source = random.choice(rettrans)
        msg_recived_list.append(source)
        #print("hi i am source",source)
        
    else:
      #print("plase find a method to do it")
      new_s = s_one_hops
      rettrans1 = []
      for i in new_s:
        one_h = get_one_hops(i,dict1)
        trans = [i for i in one_h if i not in np.array(msg_recived_list)]
        if len(trans) >= 1:  
          rettrans1.extend(trans)
      if len(rettrans1)>=1:
        #print(rettrans1)
        source = random.choice(rettrans1)
        msg_recived_list.append(source)

        #break
      else:
        #print(receiver)
        source = near_source(msg_recived_list,source,dict1,random_float_array)
        #print("implement a good code for this case")
        msg_recived_list.append(source)
        #break
        
      print(len(msg_recived_list))  
      
    
    s_one_hops

    
       
    #print(receiverl)
   
          
        
      
    #print(source)  
    #plot_interface(msg_recived_list,retrans_l)
    #if len(receiverl) > 1:
    #  source = receiverl[-1]
    #print("updated source",source)
    #break
    
#print(len(retrans_l))

msg_recived_list    
  

  

100
{'[5.93932154 7.4367043 ]': [array([5.15331426, 8.02476259]), array([6.16892724, 6.87880738])], '[6.42487038 5.90394865]': [array([6.5088615 , 6.55240597]), array([6.32837738, 6.15026715]), array([6.1296842 , 6.31785485]), array([5.82069523, 6.30918979])], '[4.81289319 6.81304702]': [array([5.10535299, 6.11590554]), array([3.86712057, 7.00669342]), array([4.81469544, 6.45753893])], '[4.9382849 9.02595701]': [array([5.02412841, 8.61767805]), array([4.92278433, 9.0273102 ]), array([4.10816513, 9.35273164])], '[9.67296484 4.45097367]': [array([9.78813519, 5.21786081]), array([9.06891936, 4.30805683])], '[8.12552534 5.76005428]': [array([8.92992877, 6.23145585]), array([7.52728852, 5.51191944]), array([8.26587063, 6.12190665]), array([7.65595716, 5.41412928])], '[6.11240105 9.33036974]': [array([6.18351846, 9.36366578])], '[1.63932452 1.7841637 ]': [array([2.43072625, 1.99337627])], '[1.18196558 8.49357861]': [array([2.11437985, 8.63207406])], '[8.00341076 8.83010933]': [array([7.6167462 , 9.65969691]), array([7.4985004 , 8.79744093])], '[9.80756508 8.19242708]': [array([9.77969355, 8.70223008])], '[5.15331426 8.02476259]': [array([5.93932154, 7.4367043 ]), array([5.02412841, 8.61767805])], '[2.06446983 6.75928919]': [array([1.1691082 , 6.55871947]), array([1.57732747, 7.23224907]), array([2.18618076, 7.44694484]), array([2.46243641, 6.54003608]), array([2.48224741, 6.59330561])], '[2.29017959 9.50202025]': [array([3.00773469, 9.5747411 ]), array([1.90204199, 9.27534352]), array([2.34503474, 8.81313452]), array([2.11437985, 8.63207406]), array([2.67573705, 9.49935151])], '[5.6966349 4.73195746]': [array([6.13177093, 4.94741362]), array([6.56213886, 4.85891831])], '[3.38100051 7.9681032 ]': [], '[5.10535299 6.11590554]': [array([4.81289319, 6.81304702]), array([4.81469544, 6.45753893]), array([5.82069523, 6.30918979])], '[1.1691082 6.55871947]': [array([2.06446983, 6.75928919]), array([1.57732747, 7.23224907])], '[6.5088615 6.55240597]': [array([6.42487038, 5.90394865]), array([7.00090044, 7.03574083]), array([6.32837738, 6.15026715]), array([6.1296842 , 6.31785485]), array([6.16892724, 6.87880738]), array([5.82069523, 6.30918979])], '[9.49373271 7.13638269]': [array([9.79084979, 6.44360968]), array([8.93561826, 7.23278431]), array([9.60475271, 6.79591179]), array([9.40792598, 6.5256936 ])], '[4.23557111 4.93328758]': [array([3.83885516, 4.27339694]), array([3.86184861, 4.72836695])], '[7.27868076 1.54202924]': [array([6.90696631, 2.24364656]), array([7.65337221, 1.35269013]), array([7.33972962, 1.28655037])], '[7.00090044 7.03574083]': [array([6.5088615 , 6.55240597]), array([6.16892724, 6.87880738])], '[2.89344305 2.16033668]': [array([2.8798908 , 2.45178566]), array([2.43072625, 1.99337627]), array([3.54526266, 2.08176905]), array([3.66526178, 2.06854947]), array([3.60465484, 2.64872226]), array([2.79996872, 1.16669615])], '[3.83885516 4.27339694]': [array([4.23557111, 4.93328758]), array([3.86184861, 4.72836695])], '[6.13177093 4.94741362]': [array([5.6966349 , 4.73195746]), array([6.56213886, 4.85891831]), array([6.86892943, 4.88276592])], '[9.89536454 1.9184033 ]': [], '[2.8798908 2.45178566]': [array([2.89344305, 2.16033668]), array([2.43072625, 1.99337627]), array([3.54526266, 2.08176905]), array([3.66526178, 2.06854947]), array([3.60465484, 2.64872226]), array([3.04673165, 3.28920834])], '[6.87797493 3.27962442]': [array([6.09941309, 3.38850542]), array([7.10034883, 3.43007176]), array([7.29531348, 3.67693256]), array([6.94156184, 3.61069846]), array([6.1950573 , 3.14103539]), array([7.57109827, 3.80750496])], '[5.19679696 3.19983033]': [array([6.09941309, 3.38850542]), array([6.1950573 , 3.14103539]), array([4.58398956, 2.88859374])], '[2.43072625 1.99337627]': [array([1.63932452, 1.7841637 ]), array([2.89344305, 2.16033668]), array([2.8798908 , 2.45178566]), array([2.79996872, 1.16669615])], '[6.90696631 2.24364656]': [array([7.27868076, 1.54202924]), array([7.56991506, 2.5446671 ])], '[2.76924126 4.31852654]': [arra

In [13]:
##### NUmber of Forwarding Nodes ########
print(len(forwarding_nodes))

55

In [14]:
##### ploting the re-transmission Nodes on the plane with red color ##############
colors = (0,0,0)
area = np.pi*3
plt.scatter(random_float_array[:,0],random_float_array[:,1], alpha=0.5)
plt.title('Flooding using RBF algorithm with retransmission nodes in red color')
plt.xlabel('x')
plt.ylabel('y')
for i in retrans_l:
  
  for j in i:
    print(j)
    plt.scatter(j[0],j[1], alpha=0.5, color='red')

In [15]:
###### Forwarding Ratio Graph ######
"""
To generate this graph please update the number of nodes in the main function above and compute the number of forwarding nodes, its a plot between number of nodes and number of forwarding nodes the formulae forawading ratio is given the paper cites, MUltiple times are need to run and store those values as in lists given below list_x and list_y


"""
list_x = [0,100,200,400]
list_y = [0,57,55,55]
plt.plot(list_x,list_y, alpha=0.5)
plt.scatter(list_x[1:],list_y[1:])
plt.title('Flooding using RBF algorithm fowarding ratio vs num of nodes')
plt.xlabel('number of nodes')
plt.ylabel('forwarding ratio')
plt.legend(['forwarding nodes','number of nodes'])